## A Machine Learning Approach to Combine the Trend-following and Counter-trend Trading Strategies

### Signal preparation file: Generate a series of X variables from raw data for our model, namely:
* Moving average, moving vol, daily price range (with different time windows)
* Normalize indicators in (1) and get their z-values (with different time windows)
* Macro data: CPI, GDP, civilian unemployment rate, FED fund rate, yield spread (T10Y2Y)
<br>*However, macro data are much lower in frequency compared to daily S&P data, and they proved to contribute little to prediction accuracy in our trials, so macro data are dropped in the final prediction model.*

### Import Packages

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Imputer
from sklearn.decomposition import PCA
from sklearn.metrics import make_scorer

### Functions: Read & Clean Data

In [2]:
def standardizeTimeString(timeString):
    if '-' in timeString:
        timeStringList = timeString.split('-')
    elif '/' in timeString:
        timeStringList = timeString.split('/')
    add_zero = '0' if len(timeStringList[1])==1 else ''
    add_zero2 = '0' if len(timeStringList[2])==1 else ''
    output = '/'.join([timeStringList[0], add_zero + timeStringList[1], add_zero2 + timeStringList[2]])
    return output

In [3]:
def read_and_clean_df(filename):
    df = pd.read_csv(filename)
    original_length = len(df)
    df.dropna(inplace=True)
    df.reset_index(drop=True,inplace=True)
    cleaned_length = len(df)
    if 'Date' in df.columns:
        df['Date'] = df['Date'].apply(lambda x: standardizeTimeString(x)) 
        if len(set(list(df['Date']))) != len(list(df['Date'])):
            print("Alert: duplicate dates discovered, not handled yet.")
    print("%d rows of data is read\n%d rows are cleaned\n%d rows remained\nThere are %d columns" % (original_length,original_length-cleaned_length,cleaned_length,len(df.columns)))
    return df

In [4]:
sp_future = read_and_clean_df('S&P_Futures_data.csv')

5515 rows of data is read
0 rows are cleaned
5515 rows remained
There are 5 columns


In [5]:
df = sp_future.copy()

### Functions: Calculate Daily Returns, Future N-Day Returns, Previous N-Day Returns

In [6]:
def calEndOfDayRet(df,col = 'Close'):
    df['past_Close'] = df[col].shift(1)
    df['eodRet'] = df.apply(lambda row: row['Close']/row['past_Close']-1, axis = 1)
    df.drop('past_Close',axis = 1,inplace = True)

In [7]:
calEndOfDayRet(df)

In [8]:
def calFret(df,period,col = 'Close'):
    df['future_Close'] = df[col].shift(-period)
    df['fret'+str(period)] = df.apply(lambda row: row['future_Close']/row['Close']-1, axis = 1)
    df.drop('future_Close',axis = 1,inplace = True)

In [9]:
def calPret(df,period,col = 'Close'):
    df['past_Close'] = df[col].shift(period+1)
    df['yesterday_Close'] = df[col].shift(1)
    df['pret'+str(period)] = df.apply(lambda row: row['yesterday_Close']/row['past_Close']-1, axis = 1)
    df.drop('yesterday_Close',axis = 1,inplace = True)
    df.drop('past_Close',axis = 1,inplace = True)

In [10]:
for p in [1,3,5,10,20,30,60,100]:
    #calFret(df,p,col = 'Close')
    calPret(df,p,col = 'Close')

### Functions: Calculate SMA, EMA

In [11]:
def calSMA(df,period,col = 'Close'):
    df['SMA_'+str(period)] = df[col].rolling(window=period).mean()

#   k = numerator/(period + 1) #weighting_multiplier
#   EMA_today = EMA_yesterday*(1-k) + current_price*k

def calEMA(df,period,col = 'Close',numerator = 2):
    df.reset_index()
    k = numerator/(period + 1) #weighting_multiplier
    col_name = 'EMA_'+str(period)
    df[col_name]= np.nan
    # Initialize the first mean
    df.loc[period-1,col_name] = df.loc[range(0,period),col].mean()    
    # EMA calculation
    for i in range(len(df[col])-period):
        df.loc[period+i,col_name] = df.loc[period+i-1,col_name]*(1-k) + df.loc[period+i,col]*k

### Functions: Calculate Sortino Ratio

In [12]:
def calMV(df,period,col = 'Close'):
    df['MV_'+str(period)] = df[col].rolling(window=period).std()
    
def calSortinoMV(df,period,col = 'Close'):
    whole_data = list(df[col])
    result_list = []
    for i in range(period-1):
        result_list.append(np.nan)
    for k in range(len(df)-(period-1)):
        neg_ret = [ret for ret in whole_data[k:k+period] if ret < 0] 
        SortinoMV = np.std(neg_ret, ddof=1)
        result_list.append(SortinoMV)
    
    df['Sortino_MV_'+str(period)] = df[col].rolling(window=period).std()
    return result_list

In [13]:
for p in [10,30,60,100,150,260]:
    calSMA(df,p,col = 'Close')
    calEMA(df,p,col = 'Close')    

In [14]:
for p in [10,30,60,100,150,260]:
    calMV(df,p,col = 'eodRet')
    calSortinoMV(df,p,col = 'eodRet')

### Function: Calculate Previous Day Range

In [15]:
def calPreviousDayrange(df,period):
    df['Dayrange'] = df.apply(lambda row: row['High'] - row['Low'],axis = 1)
    df['temp'] = df['Dayrange'].rolling(window=period).mean()
    df["Prev_"+str(period)+"_AvDayrange"] = df['temp'].shift(1)
    df.drop('temp',axis = 1,inplace = True)
    
for p in [10,20,30]:
    calPreviousDayrange(df,p)

In [16]:
df.head()

,Date,Open,High,Low,Close,eodRet,pret1,pret3,pret5,pret10,...,MV_100,Sortino_MV_100,MV_150,Sortino_MV_150,MV_260,Sortino_MV_260,Dayrange,Prev_10_AvDayrange,Prev_20_AvDayrange,Prev_30_AvDayrange
0,1997/09/10,933.75,934.25,916.75,915.25,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17.50,NaN,NaN,NaN
1,1997/09/11,916.00,917.50,900.50,908.50,-0.007375,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17.00,NaN,NaN,NaN
2,1997/09/12,908.00,926.25,904.00,924.00,0.017061,-0.007375,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,22.25,NaN,NaN,NaN
3,1997/09/15,924.75,930.00,919.50,921.75,-0.002435,0.017061,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.50,NaN,NaN,NaN
4,1997/09/16,922.50,949.25,921.00,946.25,0.026580,-0.002435,0.007102,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,28.25,NaN,NaN,NaN


In [19]:
for col in df.columns:
    print(col)

Date
Open
High
Low
Close
eodRet
pret1
pret3
pret5
pret10
pret20
pret30
pret60
pret100
SMA_10
EMA_10
SMA_30
EMA_30
SMA_60
EMA_60
SMA_100
EMA_100
SMA_150
EMA_150
SMA_260
EMA_260
MV_10
Sortino_MV_10
MV_30
Sortino_MV_30
MV_60
Sortino_MV_60
MV_100
Sortino_MV_100
MV_150
Sortino_MV_150
MV_260
Sortino_MV_260
Dayrange
Prev_10_AvDayrange
Prev_20_AvDayrange
Prev_30_AvDayrange


In [20]:
len(df)

5515

In [18]:
to_be_normalized_cols = df.columns[1:len(df.columns)]

In [ ]:
# df['Date'] = pd.to_datetime(df['Date'])
# df.set_index('Date', inplace=True)

### Functions: Calculate Z-Scores of Pre-Calculated Indicators

In [22]:
def zCol(df,col,norm_period = 20):
    if 'z_' in col:
        return 0
    df['temp_base_mean'] = df[col].rolling(window=norm_period).mean()
    df['temp_base_mean'] = df['temp_base_mean'].shift(1)
    df['temp_base_std'] = df[col].rolling(window=norm_period).std()
    df['temp_base_std'] = df['temp_base_std'].shift(1)
    df['z_'+col] = df.apply(lambda row: (row[col] - row['temp_base_mean'])/row['temp_base_std'] ,axis = 1)

print(df.columns)

for col in to_be_normalized_cols:
    zCol(df,col)
df.drop(['temp_base_mean','temp_base_std'],axis = 1,inplace = True)

def zzCol(df,col,norm_period = 60):
    if 'z_' in col:
        return 0
    df['temp_base_mean'] = df[col].rolling(window=norm_period).mean()
    df['temp_base_mean'] = df['temp_base_mean'].shift(1)
    df['temp_base_std'] = df[col].rolling(window=norm_period).std()
    df['temp_base_std'] = df['temp_base_std'].shift(1)
    df['zz_'+col] = df.apply(lambda row: (row[col] - row['temp_base_mean'])/row['temp_base_std'] ,axis = 1)

print(df.columns)

for col in to_be_normalized_cols:
    zzCol(df,col)
df.drop(['temp_base_mean','temp_base_std'],axis = 1,inplace = True)

def zzzCol(df,col,norm_period = 260):
    if 'z_' in col:
        return 0
    df['temp_base_mean'] = df[col].rolling(window=norm_period).mean()
    df['temp_base_mean'] = df['temp_base_mean'].shift(1)
    df['temp_base_std'] = df[col].rolling(window=norm_period).std()
    df['temp_base_std'] = df['temp_base_std'].shift(1)
    df['zzz_'+col] = df.apply(lambda row: (row[col] - row['temp_base_mean'])/row['temp_base_std'] ,axis = 1)

print(df.columns)

for col in to_be_normalized_cols:
    zzzCol(df,col)
df.drop(['temp_base_mean','temp_base_std'],axis = 1,inplace = True)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'eodRet', 'pret1', 'pret3',
       'pret5', 'pret10', 'pret20', 'pret30', 'pret60', 'pret100', 'SMA_10',
       'EMA_10', 'SMA_30', 'EMA_30', 'SMA_60', 'EMA_60', 'SMA_100', 'EMA_100',
       'SMA_150', 'EMA_150', 'SMA_260', 'EMA_260', 'MV_10', 'Sortino_MV_10',
       'MV_30', 'Sortino_MV_30', 'MV_60', 'Sortino_MV_60', 'MV_100',
       'Sortino_MV_100', 'MV_150', 'Sortino_MV_150', 'MV_260',
       'Sortino_MV_260', 'Dayrange', 'Prev_10_AvDayrange',
       'Prev_20_AvDayrange', 'Prev_30_AvDayrange'],
      dtype='object')


In [25]:
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)

In [ ]:
# basic calculation and output finished

### Functions: Construct Trend-following Strategies (with different parameters)

In [27]:
def create_FS_ret(df,F,S,MA_type = 'SMA',col = 'Close'):
    F_str = str(F)
    S_str = str(S)
    f_col = MA_type +"_"+ F_str
    s_col = MA_type +"_"+ S_str
    
    if f_col not in df.columns:
        if MA_type == 'SMA':
            calSMA(df,F,col)
        elif MA_type == 'EMA':
            calEMA(df,F,col)
        else:
            print("Illegal MA type!")
        
    if s_col not in df.columns:
        if MA_type == 'SMA':
            calSMA(df,S,col)
        elif MA_type == 'EMA':
            calEMA(df,S,col)
        else:
            print("Illegal MA type!")
    
    signal_col = MA_type+"_"+F_str+"_"+S_str+"_Signal"
    df[signal_col] = df.apply(lambda row : 1 if (row[f_col] > row[s_col]) else -1 ,axis = 1)
    df[signal_col] = df[signal_col].shift(1)
    
    calEndOfDayRet(df) # create column end of day Return
    
    return_col = MA_type+"_"+F_str+"_"+S_str+"_Return"
    df[return_col] = df.apply(lambda row: row[signal_col]*row['eodRet'] ,axis = 1)
    # df["Reverse_"+return_col] = df.apply(lambda row: (-1)*row[signal_col]*row['eodRet'] ,axis = 1)

    # signal_col is long/short decision for today based on information collected on the previous day
    # eodRet is today's close price over yesterday's close price, namely the return from crossover strategy if we are in today
    
    # df.drop(signal_col,axis = 1,inplace = True)
    
    return df

In [28]:
fs_choices = [10,30,60,100,150,260]
while len(fs_choices) > 0:
    f = fs_choices.pop(0)
    for s in fs_choices:
        print("f,s: ",f,s)
        df = create_FS_ret(df,f,s,'SMA')
        df = create_FS_ret(df,f,s,'EMA')    

f,s:  10 30
f,s:  10 60
f,s:  10 100
f,s:  10 150
f,s:  10 260
f,s:  30 60
f,s:  30 100
f,s:  30 150
f,s:  30 260
f,s:  60 100
f,s:  60 150
f,s:  60 260
f,s:  100 150
f,s:  100 260
f,s:  150 260


In [29]:
df.dropna(inplace = True)

### Functions: Construct Counter-trend Strategies (with different parameters)

In [31]:
df['Prev_high'] = df['High'].shift(1)
df['Prev_low'] = df['Low'].shift(1)

In [32]:
def create_CT_ret(df,period,retracement): 
    # for long Counter-trend strategy
    df["Hit_level"] = df.apply(lambda row: row['Prev_high'] - row["Prev_"+str(period)+"_AvDayrange"]*retracement, axis = 1)
    df["Hit?"] = df.apply(lambda row: 1 if row['Low'] < row["Hit_level"] else 0 , axis = 1)
    df["HitAt"] = df.apply(lambda row: row['Open']*row['Hit?'] if row['Open'] < row["Hit_level"] else row["Hit_level"]*row['Hit?'] , axis = 1)
    df["ExitPr"] = df.apply(lambda row: row["Close"]*row['Hit?'] , axis = 1)    
    df["Long_"+str(period)+"_"+str(retracement)+"_Return"] = df.apply(lambda row: (row["ExitPr"]/row['HitAt']-1) if row['Hit?'] == 1 else 0 , axis = 1)
    
    df["Hit?"+"_long_"+str(period)+"_"+str(retracement)] = df["Hit?"]
    #df["Long_short_"+str(period)+"_"+str(retracement)+"_Return"] = df.apply(lambda row: (-1)*(row["ExitPr"]/row['HitAt']-1) if row['Hit?'] == 1 else 0 , axis = 1)
    # for short Counter-trend strategy
    df["Hit_level"] = df.apply(lambda row: row['Prev_low'] + row["Prev_"+str(period)+"_AvDayrange"]*retracement, axis = 1)
    df["Hit?"] = df.apply(lambda row: 1 if row['High'] > row["Hit_level"] else 0 , axis = 1)
    df["HitAt"] = df.apply(lambda row: row['Open']*row['Hit?'] if row['Open'] > row["Hit_level"] else row["Hit_level"]*row['Hit?'] , axis = 1)
    df["ExitPr"] = df.apply(lambda row: row["Close"]*row['Hit?'] , axis = 1)    
    df["Short_"+str(period)+"_"+str(retracement)+"_Return"] = df.apply(lambda row: (row["ExitPr"]/row['HitAt']-1) if row['Hit?'] == 1 else 0 , axis = 1)
    
    df["Hit?"+"_short_"+str(period)+"_"+str(retracement)] = df["Hit?"]
    #df["Short_short_"+str(period)+"_"+str(retracement)+"_Return"] = df.apply(lambda row: (-1)*(row["ExitPr"]/row['HitAt']-1) if row['Hit?'] == 1 else 0 , axis = 1)
      
#     print("CT_"+str(period)+"_"+str(retracement)+"which---------")
#     # days of dayrange, then value of retracement
#     df["CT"+"_"+str(period)+"_"+str(retracement)+"_Return"] = df.apply(lambda row: , axis = 1)
    
    df.drop(['Hit_level','Hit?','HitAt','ExitPr'],axis = 1, inplace = True)

In [33]:
for p in [10,20,30]:
    for r in [x/10 for x in range(6,30,4)]:
        create_CT_ret(df,p,r)

In [34]:
df.drop(['Prev_high','Prev_low'],axis = 1,inplace = True)

In [35]:
df.dropna(inplace = True)

In [37]:
df.head()

,Date,Open,High,Low,Close,eodRet,pret1,pret3,pret5,pret10,...,Short_30_1.8_Return,Hit?_short_30_1.8,Long_30_2.2_Return,Hit?_long_30_2.2,Short_30_2.2_Return,Hit?_short_30_2.2,Long_30_2.6_Return,Hit?_long_30_2.6,Short_30_2.6_Return,Hit?_short_30_2.6
1,1999/10/05,1318.75,1331.25,1296.75,1315.7,-0.002275,0.019482,0.030315,0.017358,-0.022316,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
2,1999/10/06,1315.75,1337.75,1312.50,1334.7,0.014441,-0.002275,0.013480,0.016534,-0.006044,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
3,1999/10/07,1335.50,1340.75,1324.50,1329.0,-0.004271,0.014441,0.031852,0.042816,0.009759,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
4,1999/10/08,1328.75,1348.25,1318.25,1346.5,0.013168,-0.004271,0.007811,0.023725,0.029435,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
5,1999/10/11,1346.75,1350.50,1342.00,1347.0,0.000371,0.013168,0.023410,0.040974,0.045014,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0


In [38]:
len(df)

4994

### Write out the csv file (as the data used in further analysis)

In [39]:
df.to_csv('strat_returns_with_signals_new.csv',index = False)